In [ ]:
!ls

In [ ]:
#%pip install fredapi

In [ ]:
from fredapi import Fred
import pandas as pd

API_KEY = open("/home/reggie/Dropbox/API_KEYS/FRED-API-KEY","r").readlines()[0].strip()
API_KEY

In [ ]:
fred = Fred(api_key=API_KEY)
data = fred.get_series('SP500')
data

In [ ]:
MAX_SERIES = 154

In [ ]:
df = fred.search_by_category(1)
for i in range(2,MAX_SERIES+1):
    try:
        df_ = fred.search_by_category(i)
        df = pd.concat([df, df_])
        print(df.shape)
    except ValueError:
        print("No series ", i)
        continue

In [ ]:
df_monthly = df[df["frequency"]=="Monthly"]
print(df_monthly.shape)
df_monthly.head()

In [ ]:
df_monthly_nsa = df_monthly[df_monthly["seasonal_adjustment_short"]=="NSA"]
print(df_monthly_nsa.shape)

In [ ]:
s = list(df_monthly["id"])[0]
print(s)
s_df = fred.get_series_first_release(s)
s_df = s_df.reset_index()
s_df.columns = ["date",s]
s_df.head()

In [ ]:
s = list(df_monthly_nsa["id"])[0]
print(s)
s_df = fred.get_series_first_release(s)
s_df = s_df.reset_index()
s_df.columns = ["date",s]
print(s_df.shape)

for s in list(df_monthly_nsa["id"])[1:]:
    print(s)
    s_df_ = fred.get_series_first_release(s)
    s_df_ = s_df_.reset_index()
    s_df_.columns = ["date",s]
    s_df = pd.merge(s_df, s_df_)
    print(s_df.shape)

In [ ]:
print(s_df.shape)
s_df.head()


In [ ]:
sp500 = fred.get_series("SP500")
sp500 = sp500.reset_index()
sp500.columns = ['date', 'sp500']
print(sp500.shape)
sp500.head()

In [ ]:
x = pd.merge(s_df, sp500)
print(x.shape)
x.head()

In [ ]:
x['sp500_lead1'] = x['sp500'].shift(-1)
x.head()

In [ ]:
x.to_csv("../data/monthly_sp500_modeling.csv", index=False)